In [1]:
#####短信データのスクレイピング#####
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
from urllib.request import urlopen
import pandas as pd
import numpy as np
import os
import time
import itertools
import MySQLdb as sql
import mysql.connector
import requests
import re

In [2]:
##企業データの読み込みとクレンジング
#データの読み込み
company_info1 = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
index = np.where(np.in1d(company_info1["業種"], np.array(["ETF", "ETN", "REIT", "カントリーF", "インフラF"]))==False)[0]
company_info1 = company_info1.iloc[index]
company_info1.index = np.arange(company_info1.shape[0])
company_info2 = pd.read_csv("D:/Statistics/data/finance_data/stocklist.csv")
company_info2 = company_info2.rename(columns={"銘柄コード": "コード", "市場名": "取引市場", "業種分類": "業種"})
index = np.where(np.in1d(company_info2["業種"], np.array(["REIT銘柄一覧"]))==False)[0]
company_info2 = company_info2.iloc[index]
company_info2.index = np.arange(company_info2.shape[0])

#データの結合
company_info1 = company_info1[["コード", "銘柄名", "取引市場", "業種"]]
company_info2 = company_info2[["コード", "銘柄名", "取引市場", "業種"]]
company_info = pd.concat((company_info1, company_info2), axis=0)
company_info.index = np.arange(company_info.shape[0])

#変数を共通化
company = company_info[["コード", "銘柄名"]].iloc[np.where(company_info[["コード"]].duplicated()==False)[0]]
company_info["銘柄名"] = pd.merge(company_info[["コード"]], company, on="コード", how="left")["銘柄名"]
industry = np.array([['水産・農林業', '卸売業', '建設業', '非鉄金属', '鉱業', '機械', 'サービス業', '金属製品',
                      '情報・通信', '食料品', '医薬品', '不動産業', '陸運業', 'その他金融業', '小売業', 'その他製品',
                      '繊維製品', '電気機器', 'ガラス・土石製品', '証券業', '輸送用機器', '石油・石炭製品', '化学',
                      'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行業', '保険業', '倉庫・運輸関連業', '海運業',
                      '空運業', '電気・ガス業'],
                   ['水産・農林', '卸売', '建設業', '非鉄金属', '鉱業', '機械', 'サービス', '金属製品',
                    '情報通信', '食料品', '医薬品', '不動産', '陸運', 'その他金融', '小売', 'その他製品', 
                    '繊維製品', '電気機器', 'ガラス土石', '証券・先物', '輸送用機器', '石油・石炭', '化学',
                    'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行', '保険', '倉庫・運輸', '海運', 
                    '空運', '電気・ガス']])
industry_vec = np.array(company_info["業種"])
for j in range(industry.shape[1]):
    index = np.where(industry_vec==industry[0, j])[0]
    industry_vec[index] = np.array(industry[1, j], dtype="object")
company_info["業種"] = industry_vec
market = np.array([['東証1部', 'JQS', 'JQG', 'マザーズ', '東証2部', '福証Q', '名証2部', '札証', '福証',
                    '名証1部', '札証A', '名証C', '東証'],
                   ['東証1部', '東証JQS', '東証JQG', 'マザーズ', '東証2部', '福岡Q', '名証2部', '札証', "福証",
                    '名証1部',  '札幌ア', '名古屋セ', '東証']])
market_vec = np.array(company_info["取引市場"])
for j in range(market.shape[1]):
    index = np.where(market_vec==market[0, j])[0]
    market_vec[index] = np.array(market[1, j], dtype="object")
company_info["取引市場"] = market_vec

#重複を削除
company_info = company_info.iloc[np.where(company_info["コード"].duplicated()==False)[0]]
company_info.index = np.arange(company_info.shape[0])

In [3]:
##短信データを取得するための情報をスクレイピング
#待ち時間を設定
second = np.arange(1, 5)
prob = np.repeat(1.0/second.shape[0], second.shape[0])

#データの格納用配列
code = np.array(company_info["コード"])
n = company_info.shape[0]
ticker_list = [i for i in range(n)]
company_list = [i for i in range(n)]
info_list = [i for i in range(n)]

In [ ]:
#すべての企業の情報を取得
for i in range(n):
    print([i, company_info["銘柄名"].iloc[i]])
    
    #URLを取得
    url = "http://www.kabupro.jp/yuho/{code}.htm"
    r = requests.get(url.format(code=code[i]))

    #短信のリンクを取得
    r.encoding = r.apparent_encoding
    html_text = r.text
    get_data = re.findall("[有価証券報告書|四半期報告書].+a href=.+日", html_text)

    #テキストを分解して必要情報を取得
    m = len(get_data)
    report = np.array(np.repeat("", m), dtype="object")
    season = np.array(np.repeat("", m), dtype="object")
    start_period = np.array(np.repeat("", m), dtype="object")
    end_period = np.array(np.repeat("", m), dtype="object")
    url = np.array(np.repeat("", m), dtype="object")
    for j in range(m):
        report0 = re.search(".+報告書", get_data[j])
        season0 = re.search("第.+期", get_data[j])
        period0= re.search("平成.+", get_data[j])
        url0 = re.search("a href=.+htm", get_data[j])
        if report0 is not None:
            report[j] = report0[0]
        if season0 is not None:
            season[j] = season0[0]
        if period0 is not None:
            split_period = re.split("-", period0[0])
            start_period[j] = split_period[0]
            end_period[j] = split_period[1]
        if url0 is not None:
            url[j] = "http://www.kabupro.jp" + url0[0][9:]
    ticker0 = np.repeat(code[i], m)
    company0 = np.array(company_info[["銘柄名", "取引市場", "業種"]].iloc[np.repeat(i, m)])
    info0 = np.vstack((report, season, start_period, end_period, url)).T

    #リストに格納
    ticker_list[i] = ticker0
    company_list[i] = company0
    info_list[i] = info0
    
    #待ち時間
    weight_time = np.dot(np.random.multinomial(1, prob, 1), second)
    time.sleep(weight_time)

[0, '極洋']
[1, '日本水産']
[2, 'マルハニチロ']
[3, 'ホウスイ']
[4, 'カネコ種苗']
[5, 'サカタのタネ']
[6, 'ホクト']
[7, '秋川牧園']
[8, 'アクシーズ']
[9, 'ホーブ']
[10, 'ベルグアース']
[11, 'ホクリヨウ']
[12, 'ルーデン・HD']
[13, 'エムビーエス']
[14, 'ウエストHD']
[15, 'サムシングHD']
[16, '桧家HD']
[17, 'ショーボンドHD']
[18, 'ミライト・HD']
[19, 'インターライフHD']
[20, 'タマホーム']
[21, 'サンヨーホームズ']
[22, '日本アクア']
[23, 'ファーストCorp.']
[24, 'エスケーホーム']
[25, 'ベステラ']
[26, 'ＪＥＳＣＯHD']
[27, 'インベスターズクラウド']
[28, 'フィット']
[29, '岐阜造園']
[30, '安江工務店']
[31, '中外鉱業']
[32, '住石HD']
[33, '日鉄鉱業']
[34, '三井松島産業']
[35, '国際石油開発帝石']
[36, '日本海洋掘削']
[37, '石油資源開発']
[38, 'Ｋ＆Ｏエナジーグループ']
[39, '省電舎HD']
[40, 'ダイセキ環境ソリューション']
[41, '第一カッター興業']
[42, '明豊ファシリティワークス']
[43, '美樹工業']
[44, '安藤ハザマ']
[45, '東急建設']
[46, 'コムシスHD']
[47, 'ミサワホーム']
[48, '日本電技']
[49, 'シンクレイヤ']
[50, 'ビーアールHD']
[51, 'ミサワホーム中国']
[52, '麻生フオームクリート']
[53, '北弘電社']
[54, 'オーテック']
[55, '三井金属エンジニアリング']
[56, 'ＮＩＴＴＯＨ']
[57, 'シード平和']
[58, 'コーアツ工業']
[59, 'クレアHD']
[60, '太洋基礎工業']
[61, '高松コンストラクショングループ']
[62, '工藤建設']
[63, '東建Corp.']
[64, 'ソネック']
[65, '藤田エンジニアリング']
[66

[523, 'セイヒョー']
[524, 'ヨコレイ']
[525, '東洋水産']
[526, 'ジェーシー・コムサ']
[527, '日東ベスト']
[528, 'イートアンド']
[529, '大冷']
[530, 'ヨシムラ・フード・HD']
[531, '日本食品化工']
[532, '石井食品']
[533, '日清食品HD']
[534, '永谷園HD']
[535, '石垣食品']
[536, '太陽化学']
[537, 'シノブフーズ']
[538, '一正蒲鉾']
[539, 'オーケー食品工業']
[540, 'あじかん']
[541, 'フジッコ']
[542, 'ロック・フィールド']
[543, '旭松食品']
[544, 'ＪＴ']
[545, 'ケンコーマヨネーズ']
[546, '仙波糖化工業']
[547, '大森屋']
[548, 'わらべや日洋HD']
[549, 'マルタイ']
[550, 'なとり']
[551, 'サトウ食品工業']
[552, 'イフジ産業']
[553, 'ピックルスCorp.']
[554, '篠崎屋']
[555, 'ＡＦＣ-ＨＤアムスライフサイエンス']
[556, 'ＲＩＺＡＰグループ']
[557, 'ファーマフーズ']
[558, '北の達人Corp.']
[559, 'ユーグレナ']
[560, '片倉工業']
[561, 'グンゼ']
[562, 'ヒューリック']
[563, '神栄']
[564, '価値開発']
[565, 'バナーズ']
[566, 'アプライド']
[567, 'パシフィックネット']
[568, 'ラサ商事']
[569, 'クリエイト']
[570, 'アルペン']
[571, 'ハブ']
[572, 'ラクーン']
[573, 'ゴルフ・ドゥ']
[574, 'クオール']
[575, 'ケイティケイ']
[576, 'アルコニックス']
[577, '神戸物産']
[578, 'ソリトンシステムズ']
[579, 'ビューティ花壇']
[580, 'セキュアヴェイル']
[581, 'カワサキ']
[582, 'ジンズ']
[583, 'ＴＲＵＣＫ-ＯＮＥ']
[584, 'ビックカメラ']
[585, 'ＤＣＭHD']
[586, 'ペッパーフードサ

[1013, '大和コンピューター']
[1014, 'ＳＲＡHD']
[1015, 'Ｍｉｎｏｒｉソリューションズ']
[1016, 'アクロディア']
[1017, 'メディアファイブ']
[1018, 'リミックスポイント']
[1019, 'システムインテグレータ']
[1020, 'ギガプライズ']
[1021, '朝日ネット']
[1022, 'ｅＢＡＳＥ']
[1023, 'アバント']
[1024, 'アドソル日進']
[1025, 'ＯＤＫソリューションズ']
[1026, 'パス']
[1027, 'ジーダット']
[1028, 'ネクストジェン']
[1029, 'フリービット']
[1030, 'コムチュア']
[1031, 'アイフリークモバイル']
[1032, 'パシフィックシステム']
[1033, 'データ・アプリケーション']
[1034, '日本テクノ・ラボ']
[1035, 'ＮＴＴデータイントラマート']
[1036, '日本一ソフトウェア']
[1037, 'サイバーコム']
[1038, 'インフォテリア']
[1039, 'アイル']
[1040, 'Ａｂａｌａｎｃｅ']
[1041, 'ラック']
[1042, 'ユビキタス']
[1043, '王子HD']
[1044, '日本製紙']
[1045, '三菱製紙']
[1046, '北越紀州製紙']
[1047, '中越パルプ工業']
[1048, '巴川製紙所']
[1049, '大王製紙']
[1050, 'ニッポン高度紙工業']
[1051, '岡山製紙']
[1052, 'ハビックス']
[1053, '阿波製紙']
[1054, 'クラウドワークス']
[1055, 'マークラインズ']
[1056, 'メディカル・データ・ビジョン']
[1057, 'ｇｕｍｉ']
[1058, 'カヤック']
[1059, 'データセクション']
[1060, 'ＡＬＢＥＲＴ']
[1061, 'シリコンスタジオ']
[1062, 'コラボス']
[1063, 'ショーケース・ティービー']
[1064, 'エムケイシステム']
[1065, 'Ａｉｍｉｎｇ']
[1066, 'モバイルファクトリー']
[1067, 'ｓＭｅｄｉｏ']
[1068, 'ＪＩＧ-ＳＡＷ'

In [186]:
i = 1312
pd.concat((pd.DataFrame(ticker_list[i]), pd.DataFrame(company_list[i]), pd.DataFrame(info_list[i])), axis=1)

,0,0,1,2,0,1,2,3,4
0,4519,中外製薬,東証1部,医薬品,四半期報告書,第109期第1四半期,平成31年1月1日,平成31年3月31日,http://www.kabupro.jp/mark/20190426/S100FOM4.htm
1,4519,中外製薬,東証1部,医薬品,有価証券報告書,第108期,平成30年1月1日,平成30年12月31日,http://www.kabupro.jp/mark/20190328/S100FGUZ.htm
2,4519,中外製薬,東証1部,医薬品,四半期報告書,第108期第3四半期,平成30年7月1日,平成30年9月30日,http://www.kabupro.jp/mark/20181026/S100ECAU.htm
3,4519,中外製薬,東証1部,医薬品,四半期報告書,第108期第2四半期,平成30年4月1日,平成30年6月30日,http://www.kabupro.jp/mark/20180727/S100DP2O.htm
4,4519,中外製薬,東証1部,医薬品,四半期報告書,第108期第1四半期,平成30年1月1日,平成30年3月31日,http://www.kabupro.jp/mark/20180427/S100CUHM.htm
5,4519,中外製薬,東証1部,医薬品,有価証券報告書,第107期,平成29年1月1日,平成29年12月31日,http://www.kabupro.jp/mark/20180322/S100CL2M.htm
6,4519,中外製薬,東証1部,医薬品,四半期報告書,第107期第3四半期,平成29年7月1日,平成29年9月30日,http://www.kabupro.jp/mark/20171030/S100BJXW.htm
7,4519,中外製薬,東証1部,医薬品,四半期報告書,第107期第2四半期,平成29年4月1日,平成29年6月30日,http://www.kabupro.jp/mark/20170731/S100AYCC.htm
8,4519,中外製薬,東証1部,医薬品,四半期報告書,第107期第1四半期,平成29年1月1日,平成29年3月31日,http://www.kabupro.jp/mark/20170428/S100A530.htm
9,4519,中外製薬,東証1部,医薬品,有価証券報告書,第106期,平成28年1月1日,平成28年12月31日,http://www.kabupro.jp/mark/20170323/S1009X6V.htm
